In [1]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, Flatten, Dense
import re
# Read the dataset
data = pd.read_csv('IMDB.csv')
x_train = data['review'].values
y_train = data['sentiment'].values
y_train = [0 if x == "positive" else 1 for x in y_train]

# Clean text
def clean_text(text):
    if type(text) != str:
        return str(text)
    return re.sub(r'[^0-9A-Za-z" "]', '', text)

x_train = [clean_text(here) for here in x_train]

# Tokenization and padding
tokenizer = Tokenizer(num_words=100000)
tokenizer.fit_on_texts(x_train)
word_index = tokenizer.word_index
x_train = tokenizer.texts_to_sequences(x_train)
max_len = max(len(sentences) for sentences in x_train)
x_train = pad_sequences(x_train, maxlen=100)

# Convert y_train to NumPy array
y_train = np.array(y_train)

# Split the data into training and validation sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

# Model Architecture
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1, output_dim=100, input_length=100))
model.add(Conv1D(kernel_size=3, filters=32))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Model Training
model.fit(x_train, y_train, epochs=5, batch_size=20, validation_data=(x_val, y_val))

# SVM Feature Extraction
feature_extraction = model.predict(x_train)

# SVM Model
from sklearn import svm
svm_model = svm.SVC()
feature_extraction = feature_extraction * 100000
svm_model.fit(feature_extraction, y_train)





Epoch 1/5


2000/2000 [==============================] - 321s 158ms/step - loss: 0.3753 - accuracy: 0.8250 - val_loss: 0.3072 - val_accuracy: 0.8673
Epoch 2/5
2000/2000 [==============================] - 304s 152ms/step - loss: 0.1134 - accuracy: 0.9582 - val_loss: 0.4134 - val_accuracy: 0.8458
Epoch 3/5
2000/2000 [==============================] - 287s 144ms/step - loss: 0.0186 - accuracy: 0.9934 - val_loss: 0.7006 - val_accuracy: 0.8399
Epoch 4/5
2000/2000 [==============================] - 286s 143ms/step - loss: 0.0112 - accuracy: 0.9963 - val_loss: 0.8349 - val_accuracy: 0.8363
Epoch 5/5
1250/1250 [==============================] - 5s 4ms/step


SVC()

In [4]:
model.save('C:\\Users\\user\\cnnsvmmodel.h5')


C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [9]:

# Extract features using the trained model
feature_extraction_test = model.predict(x_val)
feature_extraction_test = feature_extraction_test * 100000

# Use the SVM model for predictions on the test set
y_pred = svm_model.predict(feature_extraction_test)

# Evaluate the performance
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score, recall_score
accuracy = accuracy_score(y_val, y_pred)
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')
print(f'Accuracy: {accuracy * 100:.2f}%')


classification_rep = classification_report(y_val, y_pred)
print('Classification Report:\n', classification_rep)


313/313 [==============================] - 1s 4ms/step
Precision: 84.48%
Recall: 84.04%
Accuracy: 84.42%
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.85      0.85      5039
           1       0.84      0.84      0.84      4961

    accuracy                           0.84     10000
   macro avg       0.84      0.84      0.84     10000
weighted avg       0.84      0.84      0.84     10000



In [93]:
# Predict labels for the training set
y_train_pred = svm_model.predict(feature_extraction)

# Evaluate accuracy on the training set
train_accuracy = accuracy_score(y_train, y_train_pred)
print(f'Accuracy on Training Set: {train_accuracy * 100:.2f}%')


Accuracy on Training Set: 99.87%


In [8]:
from sklearn.metrics import precision_score, recall_score

# Assuming y_val and y_pred are your true labels and predicted labels
precision = precision_score(y_val, y_pred)
recall = recall_score(y_val, y_pred)

print(f'Precision: {precision * 100:.2f}%')
print(f'Recall: {recall * 100:.2f}%')


Precision: 84.48%
Recall: 84.04%


In [1]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# Compute confusion matrix
conf_matrix_train = confusion_matrix(y_val, y_pred)

# Create a heatmap for the confusion matrix
plt.figure(figsize=(8, 6))
sns.heatmap(conf_matrix_train, annot=True, fmt="d", cmap="Blues", xticklabels=['Negative', 'Positive'], yticklabels=['Negative', 'Positive'])
plt.title('Confusion Matrix - Training Set')
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.show()


NameError: name 'y_val' is not defined

# def clean_text(text):
     if type(text) != str:
        return str(text)
     text=re.sub(r'[^0-9A-Za-z" "]', '', text)
     return text.lower()

def predict_sentiment(statement, model, tokenizer, max_len):
    # Clean text
    cleaned_statement = clean_text(statement)
    #print(cleaned_statement)
    tokenizer = Tokenizer(num_words=100000)
    tokenizer.fit_on_texts([cleaned_statement])
    #word_index = tokenizer.word_index
    #print(word_index)

    # Tokenization and padding
    sequence = tokenizer.texts_to_sequences([cleaned_statement])
   # print(sequence)
    padded_sequence = pad_sequences(sequence, maxlen=100)
   # print(padded_sequence)

    # Model prediction
    feature_extraction = model.predict(padded_sequence)
    print(feature_extraction)
    feature_extraction = feature_extraction * 100000
    print(feature_extraction)# Scaling for SVM model compatibility
    prediction = svm_model.predict(feature_extraction)
    print(prediction)

    # Convert prediction to sentiment label
    sentiment = "positive" if prediction[0] == 0 else "negative"

    return sentiment
    

# Example usage
statement_to_predict = "An awful film&*^%"
predicted_sentiment = predict_sentiment(statement_to_predict, model, tokenizer, max_len)
print("Predicted Sentiment:", predicted_sentiment)